<a href="https://colab.research.google.com/github/tanvikurade/-/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 5.4 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.6.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8999999761581421

Best val_accuracy So Far: 0.9043333530426025
Total elapsed time: 00h 04m 54s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 128)       3328      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,661,898
Trainable params: 2,661,898
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1595 - accuracy: 0.9399 - val_loss: 0.2630 - val_accuracy: 0.9098
Epoch 5/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1210 - accuracy: 0.9547 - val_loss: 0.2892 - val_accuracy: 0.9167
Epoch 6/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0900 - accuracy: 0.9665 - val_loss: 0.3233 - val_accuracy: 0.9070
Epoch 7/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0675 - accuracy: 0.9749 - val_loss: 0.3753 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0539 - accuracy: 0.9799 - val_loss: 0.3719 - val_accuracy: 0.9132
Epoch 9/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0417 - accuracy: 0.9844 - val_loss: 0.4393 - val_accuracy: 0.9138
Epoch 10/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0332 - accuracy: 0.9878 - val_loss: 0.4682 - val_